<a href="https://colab.research.google.com/github/hukim1112/LLM_application/blob/main/3_Langchain_opensourceLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-dotenv
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-openai
!pip install langchain_ollama
!pip install langchain_huggingface

모델을 다운로드합니다.

In [ ]:
!huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir ./ \
  --local-dir-use-symlinks False

스크립트로 ollama를 설치합니다.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

Modelfile을 구글 드라이브에서 다운 받습니다. EEVE 모델의 프롬프트 템플릿 정보를 갖고 있습니다.

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=14uDtG4Y5l6o1sTH3MVMWkyQSuO8RiRA8' -O /content/Modelfile

ollama에서 EEVE 모델을 서빙합니다.  nohup 명령어는 프로세스를 백그라운드에서 실행합니다.

In [ ]:
!nohup ollama serve & #ollama server 실행

In [ ]:
!nohup ollama create EEVE-Korean-10.8B -f Modelfile #gguf => 모델

In [ ]:
!ollama list #현재 모델 리스트

# Ollama 활용

ollama로 모델을 실행시킨 뒤 연결해야 함.

In [ ]:
from langchain_ollama import ChatOllama

# 올바른 URL과 인증 정보 설정
llm = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    api_base_url="http://localhost:8000",  # 서버 URL
)


In [ ]:
llm.invoke("안녕")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate
output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# LangChain 표현식 언어 체인 구문을 사용합니다.
chain = prompt | llm | StrOutputParser()

# 간결성을 위해 응답은 터미널에 출력됩니다.
# 프로덕션 환경에서 애플리케이션을 배포하기 위해 LangServe를 사용할 수 있습니다.
response = chain.invoke({"topic": "deep learning"})

In [ ]:
response

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """Answer the following question in Korean.
#Question:
{question}

#Answer: """  # 질문과 답변 형식을 정의하는 템플릿
prompt = PromptTemplate.from_template(template)  # 템플릿을 사용하여 프롬프트 객체 생성

# 프롬프트와 언어 모델을 연결하여 체인 생성
chain = prompt | llm | StrOutputParser()

question = "대한민국의 수도는 어디야?"  # 질문 정의

print(
    chain.invoke({"question": question})
)  # 체인을 호출하여 질문에 대한 답변 생성 및 출력

In [ ]:
email_conversation = """안녕하세요, 저는 김민수라고 합니다. 나이는 35세이고 현재 서울에 거주하고 있습니다.
현재 APEX Solutions라는 IT 회사에서 근무하고 있으며, 연봉은 세전 기준으로 약 6,500만 원입니다.
대출을 받고자 하는 이유는 신혼집 마련을 위해 아파트를 구매하려는 목적입니다.
현재 매매하려는 아파트의 가격은 5억 원 정도이고, 제 자산 중 약 1억 5천만 원을 계약금으로 준비할 수 있습니다.
따라서 대출로 약 3억 5천만 원을 실행하려고 합니다.

저는 신용등급이 2등급으로 안정적이고, 지난 10년 동안 큰 금융 거래 문제 없이 생활해왔습니다.
현재 대출 상담을 받으려는 이유는 금리와 상환 조건을 미리 확인하고, 저에게 적합한 대출 상품을 찾고 싶어서입니다.
가능한 한 상환 기간을 20년으로 설정하고, 중간에 일부 상환이 가능하도록 설계된 상품을 찾고 싶습니다.
또, 대출 실행이 가능한 시점과 관련해 필요한 서류와 절차도 알고 싶습니다.

추가로, 제가 다니고 있는 회사에서 곧 승진 기회가 있어 연봉이 조금 더 오를 가능성도 있습니다.
이 점이 대출 한도나 조건에 긍정적인 영향을 줄 수 있을지 궁금합니다. 감사합니다.
"""

In [ ]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)


response = llm.invoke(template.invoke({"email_conversation" : email_conversation}))

In [ ]:
response

In [ ]:
import json
from langchain.schema import BaseOutputParser

# JSON Output Parser 정의
class JsonOutputParser(BaseOutputParser):
    def get_format_instructions(self) -> str:
        """JSON 형식 출력에 대한 설명을 반환"""
        return """
Please provide only the output in the following JSON format, with no additional text, comments, or extra characters:

{
  "name": "string",
  "age": "integer",
  "company": "string",
  "salary": "integer",
  "loan_amount": "integer",
  "purpose": "string"
}
"""

    def parse(self, output: str) -> dict:
        """JSON 형식의 문자열을 파싱"""
        try:
            return json.loads(output)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON format: {e}")

# Parser 생성
parser = JsonOutputParser()


In [ ]:
structured_output = parser.parse("""{
  "name": "김민수",
  "age": 35,
  "company": "APEX Solutions",
  "salary": 65000000,
  "loan_amount": 35000000,
  "purpose": "신혼집 마련을 위한 아파트 구매(5억원)"
}""")

In [ ]:
structured_output

In [ ]:
from langchain.prompts import PromptTemplate


# 템플릿 생성
template = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# JSON 형식의 출력 포맷 설정
template = template.partial(format=parser.get_format_instructions())

In [ ]:
prompt = template.invoke({"question" : "이메일 내용을 FORMAT 형식으로 생성해주세요.", "email_conversation" : email_conversation})

In [ ]:
print(prompt.text)

In [ ]:
response = llm.invoke(prompt)

In [ ]:
response

In [ ]:
structured_output = parser.parse(response.content)

In [ ]:
print(structured_output)

# vLLM 활용(A100 이상 필요)

미니프로젝트 주제 : vLLM으로 호스팅된 Huggingface 기반 한국어 모델(Llama 3)을 활용하여 LangChain으로 Chain을 설계하고, 이를 기반으로 실제 사용할 수 있는 LLM 앱을 개발합니다.

1. vLLM으로 모델을 호스팅 (필요 시 ngrok을 사용해 특정 도메인으로 api 호출이 가능하도록 할 것)

2. PromptTemplates를 디자인하고, Chain을 구성

3. LangChain으로 response를 잘 생성하는 지 확인하기

4. Gradio Chat UI에 연결하여 대화해보기.

## vLLM으로 모델 호스팅 (LangChain과 연결)

In [ ]:
!pip install --upgrade --quiet  vllm -q

모델 다운로드(수 분 이상 걸릴 수 있음) 및 vLLM 서버 구동

In [ ]:
!nohup vllm serve Qwen/Qwen2.5-7B-Instruct > nohup_script.out &

nohup: redirecting stderr to stdout


### Curl 명령어 테스트 (vLLM의 api 서버 사용)

In [ ]:
import subprocess

# curl 명령어를 Python에서 실행
command = '''curl http://localhost:8000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model": "Qwen/Qwen2.5-7B-Instruct",
    "messages": [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant. you will only respond in Korean."},
        {"role": "user", "content": "내 이름은 김형욱입니다."},
        {"role": "assistant", "content": "안녕하세요, 김형욱님! 만나서 반갑습니다. 궁금한 사항이나 도와드릴 일이 있으신가요?"},
        {"role": "user", "content" : "내 이름 기억해?"}
    ],
    "temperature": 0.7,
    "top_p": 0.8,
    "repetition_penalty": 1.05,
    "max_tokens": 512
  }' '''  # 마지막 작은따옴표 수정

# subprocess를 사용하여 curl 명령어 실행
result = subprocess.run(command, shell=True, capture_output=True, text=True)

# 결과 출력
print(result.stdout)


{"id":"chatcmpl-050715bf2e5e4c3fab4b26fceedd3146","object":"chat.completion","created":1734483160,"model":"Qwen/Qwen2.5-7B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"네, 김형욱님이라는 이름을 기억하고 있습니다. 언제든지 도움이 필요하시면 말씀해주세요.","tool_calls":[]},"logprobs":null,"finish_reason":"stop","stop_reason":null}],"usage":{"prompt_tokens":91,"total_tokens":117,"completion_tokens":26,"prompt_tokens_details":null},"prompt_logprobs":null}


### OpenAI Client를 사용한 질의요청

vLLM은 LLM과 상호작용할 때, OpenAI의 api 포맷을 그대로 지원함.

In [ ]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B-Instruct",
    messages=[
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant. you will only respond in Korean."},
        {"role": "user", "content": "내 이름은 김형욱입니다."},
        {"role": "assistant", "content": "안녕하세요, 김형욱님! 만나서 반갑습니다. 궁금한 사항이나 도와드릴 일이 있으신가요?"},
        {"role": "user", "content" : "내 이름 기억해?"}
    ],
    temperature=0.7,
    top_p=0.8,
    max_tokens=512,
    extra_body={
        "repetition_penalty": 1.05,
    },
)
print("Chat response:", chat_response.choices[0].message.content)

Chat response: 네, 김형욱님이라는 이름을 기억하고 있습니다. 언제든지 도움이 필요하시면 말씀해 주세요.


### LangChain와 vLLM 연결

- openai_api_base에 api 서버의 주소를 입력. 현재 localhost의 8000번 포트를 사용 중.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base="http://localhost:8000/v1",
    model_name="Qwen/Qwen2.5-7B-Instruct",
    temperature=0.7,
    top_p=0.8,
    frequency_penalty=1.05,
    max_tokens=512
)
print(llm.invoke("내 이름은 김형욱이라고 합니다."))

content='안녕하세요, 김형욱님! 만나서 반갑습니다. 저는 Qwen이라고 합니다. 알리바바 클라우드에서 만든 언어 모델입니다. 오늘 어떤 도움이 필요하신가요? 궁금한 점이나 이야기 나누고 싶은 주제가 있으시다면 말씀해 주세요.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 38, 'total_tokens': 112, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-0fc9834c-08fb-4fa3-87fe-e9b9a808ece9-0' usage_metadata={'input_tokens': 38, 'output_tokens': 74, 'total_tokens': 112, 'input_token_details': {}, 'output_token_details': {}}


### LangChain의 ChatOpenAI 클래스로 연결

In [ ]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

llm.invoke(
    [
        SystemMessage(content="You are Qwen, created by Alibaba Cloud. You are a helpful assistant. you will only respond in Korean."),
        HumanMessage(content="내 이름은 김형욱입니다."),
        AIMessage(content="반가워요, 오늘 어떻게 도와드릴까요?"),
        HumanMessage(content="한국의 버팀목 전세대출에 대해 설명해줘.")
    ]
)


AIMessage(content='버팀목 전세대출은 주택 구매를 돕기 위한 방법 중 하나로, 주택 구매자의 성립되지 않은 신용이나 낮은 소득으로 대출을 받기 어렵다고 판단되는 경우에 제공되는 대출입니다. 이 프로그램은 주로 저소득 가구나 신용 기록이 부족한 이들에게 전세 입주를 돕는 데 사용됩니다.\n\n버팀목 전세대출의 주요 특징은 다음과 같습니다:\n\n1. 보증금과 월세 보조: 대출은 보증금과 월세를 지원합니다. 이는 주택을 구매하는 데 필요한 초기 비용을 지원합니다.\n\n2. 정부 지원: 이 대출은 주로 정부가 지원하는 프로그램으로, 대출금 상환을 보증하거나 향후 대출금을 주택 구매자에게 반환하도록 돕는 역할을 합니다.\n\n3. 대출 조건: 대출 조건은 일반적으로 신용 점수, 소득, 재무 상황 등을 기준으로 합니다. 저소득층이나 신용 기록이 부족한 이들을 대상으로 합니다.\n\n4. 대출 기간: 대출 기간은 일반적으로 3년에서 5년 사이이며, 이 기간 동안 대출은 보증금과 월세로만 사용될 수 있습니다.\n\n5. 대출 상환: 대출은 주택을 구매한 후 신청자가 주택을 판매하거나 대출을 직접 상환할 때까지 계속됩니다.\n\n이러한 버팀목 전세대출은 주택 구매를 희망하는 저소득층이나 신용 기록이 부족한 이들에게 큰 도움이 될 수 있습니다. 하지만, 구체적인 조건과 절차는 지역별로 다르므로, 지역별로 제공되는 정확한 정보를 확인하시기 바랍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 420, 'prompt_tokens': 82, 'total_tokens': 502, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': None, 'finish_reas

### Reference

LangChain으로 원격에 있는 vLLM 서버(openAI api 스타일)로 챗봇 연결 [link](https://python.langchain.com/docs/integrations/chat/vllm/)

vLLM+LangChain [link](https://python.langchain.com/docs/integrations/llms/vllm/)

vLLM complete guide [link](https://lunary.ai/blog/vllm-langchain-tutorial)